In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

In [3]:
WORKING_DIRECTORY = './drive/MyDrive/Colab\ Notebooks/diploma/'

In [4]:
 # Создаём SparkSession
spark = SparkSession.builder \
    .appName("PetCo") \
    .getOrCreate()

In [7]:
# Read dataset with beh logs
data_items = spark.read.parquet(WORKING_DIRECTORY+'data/data_set_items')

In [9]:
data_items.show()

+----------+---------------+--------------------+-----+-----------+--------------------+--------------------+--------------+-------------+--------------------+------------+-------------------+-------------------+----------+--------------------+
|        id|autocomplete_id|                name|score|customer_id|          name_lower|       metadata_json|computed_score|boost_or_bury|    ds_metadata_json|section_name|         created_at|         updated_at|       day|              ac_key|
+----------+---------------+--------------------+-----+-----------+--------------------+--------------------+--------------+-------------+--------------------+------------+-------------------+-------------------+----------+--------------------+
|3668992957|           1947|Kaytee Forti-Diet...|   -1|    5017277|kaytee forti-diet...|{"url": "/shop/en...|         68803|            0|{"weighted_keywor...|    Products|2022-05-16 14:44:33|2024-06-26 02:09:22|2024-06-26|key_afiSr5Y4gCaaSW5X|
|3668994348|        

In [10]:
data_items.printSchema()  # Структура данных

root
 |-- id: long (nullable = true)
 |-- autocomplete_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- name_lower: string (nullable = true)
 |-- metadata_json: string (nullable = true)
 |-- computed_score: integer (nullable = true)
 |-- boost_or_bury: integer (nullable = true)
 |-- ds_metadata_json: string (nullable = true)
 |-- section_name: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- day: string (nullable = true)
 |-- ac_key: string (nullable = true)



In [11]:
print(f'Число строк: {data_items.count()}')

Число строк: 4900493


In [14]:
n_items = data_items.select('customer_id').distinct().count()
print(f'Число товаров в каталоге: {n_items}')

Число товаров в каталоге: 32001


In [17]:
one_item = data_items.filter(data_items.customer_id == 5017277)
one_item.show()

+----------+---------------+--------------------+-----+-----------+--------------------+--------------------+--------------+-------------+--------------------+------------+-------------------+-------------------+----------+--------------------+
|        id|autocomplete_id|                name|score|customer_id|          name_lower|       metadata_json|computed_score|boost_or_bury|    ds_metadata_json|section_name|         created_at|         updated_at|       day|              ac_key|
+----------+---------------+--------------------+-----+-----------+--------------------+--------------------+--------------+-------------+--------------------+------------+-------------------+-------------------+----------+--------------------+
|3668992957|           1947|Kaytee Forti-Diet...|   -1|    5017277|kaytee forti-diet...|{"url": "/shop/en...|         68803|            0|{"weighted_keywor...|    Products|2022-05-16 14:44:33|2024-06-26 02:09:22|2024-06-26|key_afiSr5Y4gCaaSW5X|
|3668992957|        

In [19]:
one_item.count()

180

In [22]:
one_item.take(1)[0]

Row(id=3668992957, autocomplete_id=1947, name='Kaytee Forti-Diet Pro Health Mouse, Rat Food', score=-1, customer_id='5017277', name_lower='kaytee forti-diet pro health mouse, rat food', metadata_json='{"url": "/shop/en/petcostore/product/kaytee-fortidiet-prohealth-ratmouse-food-5-lbs", "facets": {"Line": ["Forti-Diet Pro Health"], "form": ["Powder or Pellets or Granules"], "Organic": ["No"], "Holistic": ["No"], "Pet Type": ["Rat"], "Food Form": ["Powder or Pellets or Granules"], "Lifestage": ["All Lifestages"], "How to get it": ["Same Day Delivery", "Free Pickup Today", "One Time Delivery", "Repeat Delivery"], "Primary Brand": ["Kaytee"], "Customer Rating": [4.5395], "Killed Item flag": ["No"], "Primary Pet Type": ["Small Animal"], "Rx Food Indicator": ["No"], "Dietary Preference": ["With Grain"], "Personalized Item flag": ["No"], "PTC_OMNI_BRAND_SUB_BRND": ["Forti-Diet Pro Health"], "Limited Ingredients Diet": ["No"]}, "mfName": "Kaytee", "itemimg": "https://assets.petco.com/petco/ima

In [23]:
one_item.take(10)[9]

Row(id=None, autocomplete_id=1947, name='Kaytee Forti-Diet Pro Health Mouse, Rat Food', score=-1, customer_id='5017277', name_lower='kaytee forti-diet pro health mouse, rat food', metadata_json='{"url": "/shop/en/petcostore/product/kaytee-fortidiet-prohealth-ratmouse-food-5-lbs", "image_url": "https://assets.petco.com/petco/image/upload/f_auto,q_auto,w_190/dpr_auto/1226754-center-1", "facets": {"Line": ["Forti-Diet Pro Health"], "form": ["Powder or Pellets or Granules"], "Organic": ["No"], "Holistic": ["No"], "Pet Type": ["Rat"], "Food Form": ["Powder or Pellets or Granules"], "Lifestage": ["All Lifestages"], "How to get it": ["Same Day Delivery", "Free Pickup Today", "One Time Delivery", "Repeat Delivery"], "Primary Brand": ["Kaytee"], "Customer Rating": ["4.5395"], "Killed Item flag": ["No"], "Primary Pet Type": ["Small Animal"], "Rx Food Indicator": ["No"], "Dietary Preference": ["With Grain"], "Personalized Item flag": ["No"], "PTC_OMNI_BRAND_SUB_BRND": ["Forti-Diet Pro Health"], "